In [10]:
from transformers import AutoModel, AutoTokenizer
import torch
from model.openllama import OpenLLAMAPEFTModel

# Initialize the model
args = {
    'model': 'openllama_peft',
    'imagebind_ckpt_path': '../pretrained_ckpt/imagebind_ckpt',
    'vicuna_ckpt_path': '../pretrained_ckpt/vicuna_ckpt/converted/vicuna_full',
    'delta_ckpt_path': '../pretrained_ckpt/pandagpt_ckpt/7b/pandagpt_7b_max_len_1024/pytorch_model.pt',
    'stage': 2,
    'max_tgt_len': 128,
    'lora_r': 32,
    'lora_alpha': 32,
    'lora_dropout': 0.1,
}
print("Initializing model...")
model = OpenLLAMAPEFTModel(**args)
delta_ckpt = torch.load(args['delta_ckpt_path'], map_location=torch.device('cpu'))
model.load_state_dict(delta_ckpt, strict=False)
model = model.eval().half().cuda()
print("Model initialized.")

def generate_response(prompt_text, image_path=None, audio_path=None, video_path=None, thermal_path=None, top_p=0.01, temperature=1.0, max_length=128):
    """Generate a response from the model."""
    response = model.generate({
        'prompt': prompt_text,
        'image_paths': [image_path] if image_path else [],
        'audio_paths': [audio_path] if audio_path else [],
        'video_paths': [video_path] if video_path else [],
        'thermal_paths': [thermal_path] if thermal_path else [],
        'top_p': top_p,
        'temperature': temperature,
        'max_tgt_len': max_length,
        'modality_embeds': []

    })
    return response

Initializing model...
Initializing visual encoder from ../pretrained_ckpt/imagebind_ckpt ...
Visual encoder initialized.
Initializing language decoder from ../pretrained_ckpt/vicuna_ckpt/converted/vicuna_full ...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 33554432 || all params: 6771978240 || trainable%: 0.49548936530546206
Language decoder initialized.
Model initialized.


In [18]:
# Example usage
prompt_text = "Can you describe the image?"
image_path = "/home/tz362/Desktop/projects/adversarial_illusions/data/thermal_perturbed/001.png"  # Update this path to your image
# image_path = "/home/tz362/Desktop/projects/adversarial_illusions/data/thermal/001.jpg"  # Update this path to your image

# Generate response
response = generate_response(prompt_text, image_path=image_path)
print("Response:", response)


Response: The image shows a man standing in a dark room, holding a gun in his hand. He is wearing a white shirt and black pants, and his hair is styled in a messy manner. The man appears to be pointing the gun at something in the distance, with a sense of tension in his expression. The room has a dim lighting, and there are no other visible objects or people in the scene.


In [8]:
import os
from tqdm import tqdm

responses = {}

for filename in tqdm(os.listdir("/home/tz362/Desktop/projects/adversarial_illusions/data/thermal")):
    image_path = os.path.join("/home/tz362/Desktop/projects/adversarial_illusions/data/thermal", filename)
    prompt_text = "Answer with yes or no, does this image describe anything suspious?"
    response = generate_response(prompt_text, image_path=image_path)
    print(f"Response for {filename}:", response)
    responses[filename] = response

import json
with open("/home/tz362/Desktop/projects/adversarial_illusions/data/thermal_responses.json", "w") as f:
    json.dump(responses, f)

print("Responses saved to /home/tz362/Desktop/projects/adversarial_illusions/data/thermal_responses.json")

  1%|          | 1/100 [00:04<07:19,  4.44s/it]

Response for 001.jpg: No, the image does not describe anything suspicious. It shows a car parked in front of an apartment building, with its front right wheel slightly in the air. Additionally, there is a person walking down the street, but their actions do not raise any concerns or suspicions.


  2%|▏         | 2/100 [00:10<09:08,  5.60s/it]

Response for 002.jpg: Yes, the image depicts a woman dressed in black standing in a parking lot in front of multiple cars. The scene appears to be nighttime, with a dark background and headlights shining on the woman. Given the mysterious and potentially eerie atmosphere in the image, it can raise suspicions regarding the woman's presence in the parking lot, especially if we consider her attire and the lack of other visible people.


  3%|▎         | 3/100 [00:19<11:15,  6.96s/it]

...

Response for 021.jpg: Yes, this image describes an unusual situation. The photo shows a man riding a motorcycle through an alleyway at night with a woman who appears to be a ghost. The combination of these elements makes the image both intriguing and eerie, as it seems to depict a supernatural encounter.


 19%|█▉        | 19/100 [01:55<07:33,  5.60s/it]

Response for 016.jpg: Yes, the image depicts a suspicious scene where two women in white clothes are seen walking down a dark street at night. They are carrying backpacks, and a man in a suit is chasing after them. The dark street and the presence of the menacing man add an element of intrigue and danger to the situation, making it suspicious.


 19%|█▉        | 19/100 [02:01<08:36,  6.38s/it]


KeyboardInterrupt: 